In [1]:
using Random

push!(LOAD_PATH, ".")

using SimpleAgentEvents
using SimpleAgentEvents.Scheduler

In [2]:
@enum Status susceptible infected recovered

mutable struct Person
    status :: Status
    contacts :: Vector{Person}
    x :: Int
    y :: Int
end

Person(x, y) = Person(susceptible, [], x, y)
Person(state, x, y) = Person(state, [], x, y)

Person

In [3]:
mutable struct Simulation
    scheduler :: PQScheduler{Float64}
    a :: Float64
    b :: Float64
    pop :: Vector{Person}
end

scheduler(sim :: Simulation) = sim.scheduler

Simulation(a, b) = Simulation(PQScheduler{Float64}(), a, b, [])

Simulation

In [4]:
@processes sim person::Person begin
    @poisson(sim.a * count(p -> p.status == infected, person.contacts)) ~
        person.status == susceptible        => begin
                person.status = infected
                [person; person.contacts]
            end

    @poisson(sim.b)  ~
        person.status == infected           => begin
                person.status = recovered
                person.contacts
            end
end


process_poisson (generic function with 1 method)

In [5]:
include("setup_world.jl")

setup_grid (generic function with 1 method)

In [12]:

sim = Simulation(0.5, 0.1)
sim.pop = setup_grid(50, 50)


for person in sim.pop
    spawn(person, sim)
end

Random.seed!(42);

In [15]:

for t in  1:10
    upto!(sim.scheduler, time_now(sim.scheduler) + 1.0)
    println(time_now(sim.scheduler), ", ", 
        count(p -> p.status == infected, sim.pop), ", ",
        count(p -> p.status == recovered, sim.pop), ", ",
        count(p -> p.status == susceptible, sim.pop))
end



21.0, 205, 142, 2153
22.0, 228, 158, 2114
23.0, 247, 180, 2073
24.0, 287, 210, 2003
25.0, 297, 240, 1963
26.0, 314, 269, 1917
27.0, 331, 300, 1869
28.0, 350, 331, 1819
29.0, 358, 366, 1776
30.0, 365, 410, 1725
